In [1]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [2]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features. Use the data from 2005.


In [3]:
query = '''SELECT film_id, rental_duration, rental_rate, length, replacement_cost, rating, store_id, name FROM sakila.film f
LEFT JOIN sakila.inventory i USING (film_id)
LEFT JOIN sakila.rental r USING (inventory_id)
JOIN sakila.film_category USING(film_id)
JOIN sakila.category USING (category_id)
GROUP BY f.film_id;'''

In [4]:
data = pd.read_sql_query(query,engine)
data

,film_id,rental_duration,rental_rate,length,replacement_cost,rating,store_id,name
0,19,6,0.99,113,20.99,PG,1.0,Action
1,21,3,4.99,129,17.99,R,1.0,Action
2,29,5,2.99,168,11.99,NC-17,1.0,Action
3,38,6,0.99,68,25.99,NC-17,NaN,Action
4,56,6,2.99,129,15.99,G,1.0,Action
...,...,...,...,...,...,...,...,...
995,931,7,0.99,48,9.99,PG-13,2.0,Travel
996,977,3,2.99,85,25.99,R,2.0,Travel
997,981,7,0.99,55,13.99,NC-17,1.0,Travel
998,988,7,2.99,139,26.99,R,1.0,Travel


2. Create a query to get the list of films and a boolean indicating if it was rented last month (May 2005). This would be our target variable.


In [5]:
query2 = '''SELECT film_id, rental_duration, rental_rate, length, replacement_cost, rating, store_id, name, COUNT(DISTINCT r.rental_id) AS rented_times, COUNT(DISTINCT i.inventory_id) AS copies, 
    CASE 
        WHEN r.rental_date BETWEEN '2005-05-01' AND '2005-05-31' THEN True
        ELSE False END AS rented_in_may FROM sakila.film f
    LEFT JOIN sakila.inventory i USING (film_id)
    LEFT JOIN sakila.rental r USING (inventory_id)
    JOIN sakila.film_category USING(film_id)
    JOIN sakila.category USING (category_id)
    GROUP BY f.film_id;'''

3. Read the data into a Pandas dataframe.


In [6]:
data2 = pd.read_sql_query(query2,engine)
data2

,film_id,rental_duration,rental_rate,length,replacement_cost,rating,store_id,name,rented_times,copies,rented_in_may
0,1,6,0.99,86,20.99,PG,1.0,Documentary,24,8,0
1,2,3,4.99,48,12.99,G,2.0,Horror,7,3,0
2,3,7,2.99,50,18.99,NC-17,2.0,Documentary,12,4,0
3,4,5,2.99,117,26.99,G,1.0,Horror,23,7,0
4,5,6,2.99,130,22.99,G,2.0,Family,12,3,1
...,...,...,...,...,...,...,...,...,...,...,...
995,996,6,0.99,183,9.99,G,1.0,Documentary,7,2,0
996,997,4,0.99,179,14.99,NC-17,1.0,Music,6,2,0
997,998,6,0.99,105,10.99,NC-17,2.0,Horror,9,2,0
998,999,5,2.99,101,28.99,R,1.0,Children,17,5,0


4. Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.


In [7]:
# Xy Split
y = data2['rented_in_may']
X = data2.drop(['rented_in_may','film_id'],axis=1)

In [8]:
# converting some data
X['store_id'] = X['store_id'].astype('object')
X['store_id'].value_counts(dropna=False)

1.0    759
2.0    199
NaN     42
Name: store_id, dtype: int64

In [9]:
X['store_id'] = X['store_id'].fillna('unknown')
X['store_id'] = X['store_id'].replace(1,'store_1').replace(2,'store_2')
X['store_id'].value_counts(dropna=False)

store_1    759
store_2    199
unknown     42
Name: store_id, dtype: int64

In [10]:
# Train-test Split
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(800, 9)
(200, 9)
(800,)
(200,)


In [12]:
X_train_cat = X_train.select_dtypes(include = object)
X_train_num = X_train.select_dtypes(include = np.number)

In [13]:
# MinMax
X_train_num.describe().T

,count,mean,std,min,25%,50%,75%,max
rental_duration,800.0,4.97125,1.419663,3.00,4.00,5.00,6.00,7.00
rental_rate,800.0,2.99000,1.638095,0.99,0.99,2.99,4.99,4.99
length,800.0,115.92625,40.583582,46.00,80.75,115.00,150.00,185.00
replacement_cost,800.0,20.09125,6.039586,9.99,14.99,19.99,24.99,29.99
rented_times,800.0,16.18000,7.292913,0.00,11.00,16.00,21.00,34.00
copies,800.0,4.62875,2.013328,0.00,3.00,5.00,6.00,8.00


In [14]:
from sklearn.preprocessing import MinMaxScaler

In [15]:
MinMaxtransformer = MinMaxScaler().fit(X_train_num) 
X_train_norm = MinMaxtransformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_norm,columns=X_train_num.columns) 
X_train_norm

,rental_duration,rental_rate,length,replacement_cost,rented_times,copies
0,0.75,1.0,0.769784,0.50,0.470588,0.500
1,1.00,0.5,0.424460,0.45,0.264706,0.375
2,0.75,0.5,0.683453,0.60,0.647059,0.750
3,1.00,0.5,0.453237,0.20,0.676471,0.750
4,0.50,1.0,0.532374,0.95,0.470588,0.750
...,...,...,...,...,...,...
795,0.75,0.0,0.345324,0.70,0.000000,0.000
796,0.75,0.0,0.395683,0.90,0.235294,0.375
797,0.25,0.5,0.079137,1.00,0.205882,0.250
798,0.00,0.0,0.215827,0.50,0.235294,0.375


In [16]:
# Encoding categoricals
X_train_cat.describe().T

,count,unique,top,freq
rating,800,5,PG-13,186
store_id,800,3,store_1,615
name,800,16,Foreign,62


In [17]:
from sklearn.preprocessing import OneHotEncoder

In [18]:
X_train_cat1 = pd.DataFrame(X_train_cat['rating'])
X_train_cat2 = pd.DataFrame(X_train_cat['store_id'])
X_train_cat3 = pd.DataFrame(X_train_cat['name'])

In [19]:
encoder1 = OneHotEncoder(drop='first').fit(X_train_cat1)
print(encoder1.categories_)
transformer1 = encoder1.transform(X_train_cat1).toarray()

[array(['G', 'NC-17', 'PG', 'PG-13', 'R'], dtype=object)]


In [20]:
encoder2 = OneHotEncoder(drop='first').fit(X_train_cat2)
print(encoder2.categories_)
transformer2 = encoder2.transform(X_train_cat2).toarray()

[array(['store_1', 'store_2', 'unknown'], dtype=object)]


In [21]:
encoder3 = OneHotEncoder(drop='first').fit(X_train_cat3)
print(encoder3.categories_)
transformer3 = encoder3.transform(X_train_cat3).toarray()

[array(['Action', 'Animation', 'Children', 'Classics', 'Comedy',
       'Documentary', 'Drama', 'Family', 'Foreign', 'Games', 'Horror',
       'Music', 'New', 'Sci-Fi', 'Sports', 'Travel'], dtype=object)]


In [22]:
col_names1 = ['NC-17', 'PG', 'PG-13', 'R']
X_1_enc = pd.DataFrame(transformer1,columns=col_names1)

In [23]:
col_names2 = ['store_2', 'unknown']
X_2_enc = pd.DataFrame(transformer2,columns=col_names2)

In [24]:
col_names3 = ['Animation', 'Children', 'Classics', 'Comedy',
       'Documentary', 'Drama', 'Family', 'Foreign', 'Games', 'Horror',
       'Music', 'New', 'Sci-Fi', 'Sports', 'Travel']
X_3_enc = pd.DataFrame(transformer3,columns=col_names3)

In [25]:
# Concatenating X_train data
X_train_trans = pd.concat([X_train_norm, X_1_enc, X_2_enc,X_3_enc], axis=1)
X_train_trans.shape

(800, 27)

5. Create a logistic regression model to predict this variable from the cleaned data.

In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='multinomial').fit(X_train_trans, y_train)

In [ ]:
# transform test data with previous encoders 

In [29]:
X_test_cat = X_test.select_dtypes(include = object)
X_test_num = X_test.select_dtypes(include = np.number)

In [32]:
X_test_norm = MinMaxtransformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_norm,columns=X_test_num.columns) 

In [33]:
X_test_cat1 = pd.DataFrame(X_test_cat['rating'])
X_test_cat2 = pd.DataFrame(X_test_cat['store_id'])
X_test_cat3 = pd.DataFrame(X_test_cat['name'])

In [34]:
trans1 = encoder1.transform(X_test_cat1).toarray()
X_test1_enc = pd.DataFrame(trans1,columns=col_names1)

In [35]:
trans2 = encoder2.transform(X_test_cat2).toarray()
X_test2_enc = pd.DataFrame(trans2,columns=col_names2)

In [36]:
trans3 = encoder3.transform(X_test_cat3).toarray()
X_test3_enc = pd.DataFrame(trans3,columns=col_names3)

In [38]:
# Concatenating test data
X_test_trans = pd.concat([X_test_norm, X_test1_enc, X_test2_enc,X_test3_enc], axis=1)
X_test_trans.shape

(200, 27)

In [ ]:
# predict test data

In [40]:
predictions = classification.predict(X_test_trans)
classification.score(X_test_trans, y_test)

0.8

In [45]:
print(y_test.value_counts())
pd.Series(predictions).value_counts()

0    161
1     39
Name: rented_in_may, dtype: int64


0    199
1      1
dtype: int64

In [46]:
# confusion matrix
from sklearn.metrics import confusion_matrix

In [47]:
confusion_matrix(y_test, predictions)

array([[160,   1],
       [ 39,   0]], dtype=int64)

6. Evaluate the results.

As we saw before, the logistic regression tends to overcompensate to where it has more values, so it was very precise with one value, but very less precise with the other, I will try the KNN classifier to try get better results

In [48]:
from sklearn import neighbors

In [57]:
clf = neighbors.KNeighborsClassifier(n_neighbors=3, weights='uniform')
clf.fit(X_train_trans, y_train)
predictions_clf = clf.predict(X_test_trans)
clf.score(X_test_trans, y_test)

0.755

In [58]:
confusion_matrix(y_test, predictions_clf)

array([[147,  14],
       [ 35,   4]], dtype=int64)

In this case we got some more values from the lesser value, but less rigth values from the other one, so it would be better the logistic regression